In [1]:
import pandas as pd
import numpy as np

In [73]:
data = pd.read_csv('alz_data.csv')
data = data.drop(columns='Unnamed: 0')

In [74]:
zs = data
zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-19.1)/6.7))
zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.3)/2.3))
zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+30.9)/13.4))
zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+82.2)/46.3))
zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-26.3)/2.8))
zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-21.4)/5.7))
zs = zs.drop(['GDS', 'CDR','Delayed Recall','Number Span Test Forward','Part A Trail Making','Part B Trail Making','moca','Categories Animal'], axis = 1)

In [80]:
zs_ab = zs
zs_ab['DR_ab'] = zs_ab['Delayed_Recall_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['NSF_ab'] = zs_ab['Number_Span_Test_Forward_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['TMA_ab'] = zs_ab['Part_A_Trail_Making_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['TMB_ab'] = zs_ab['Part_B_Trail_Making_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['MoCA_ab'] = zs_ab['MoCA_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['Cat_An_ab'] = zs_ab['Categories_Animal_Z'].apply(lambda x: 1 if x <= -1.5 else 0)


In [76]:
data = data.drop(['Delayed Recall','Number Span Test Forward','Part A Trail Making','Part B Trail Making','moca','Categories Animal'], axis = 1)
data_sum = data.sum(axis = 1)

In [83]:
zs_ab = zs_ab.drop(['Delayed_Recall_Z','Number_Span_Test_Forward_Z','Part_A_Trail_Making_Z','Part_B_Trail_Making_Z','MoCA_Z','Categories_Animal_Z'], axis = 1)

In [84]:
zs_ab_sum = zs_ab.sum(axis = 1)
zs_ab_sum.columns = ['z score sum']

In [85]:
print(zs_ab_sum)

0     0
1     0
2     0
3     0
4     0
     ..
95    6
96    6
97    6
98    6
99    6
Length: 100, dtype: int64


In [16]:
data = data.join(zs_ab_sum)

ValueError: Other Series must have a name

In [8]:
def alz_diagnose(data):
    data_sum = data.sum(axis = 1)
    diags = []
    for row in range(data_sum.shape[0]):
        row_sum = data.iloc[row].sum()
        try:
            if row_sum == 1:
                diags.append('Normal')
            elif row_sum == 2:
                diags.append('SCD')
            elif row_sum == 3:
                diags.append('MCI')
            elif row_sum == 4:
                diags.append('Probable Dementia')
            elif row_sum == 5:
                diags.append('Mild Dementia')
            elif row_sum == 8:
                diags.append('Moderate Dementia')
            elif row_sum == 10:
                diags.append('Severe Dementia')
            else:
                raise ValueError()
        except ValueError:
            diags.append('This combination of diagnostic values does not indicate clear diagnosis. Please verify values are correct.')
            pass
    return diags

In [9]:
data_diag = alz_diagnose(data)
df5 = pd.DataFrame({'Diagnosis':data_diag})
data = data.join(df5)


In [10]:
print(data)

    GDS  CDR                                          Diagnosis
0     1  0.0                                             Normal
1     1  0.0                                             Normal
2     1  0.0                                             Normal
3     1  0.0                                             Normal
4     1  0.0                                             Normal
..  ...  ...                                                ...
95    6  3.0  This combination of diagnostic values does not...
96    6  3.0  This combination of diagnostic values does not...
97    7  3.0                                    Severe Dementia
98    7  3.0                                    Severe Dementia
99    7  3.0                                    Severe Dementia

[100 rows x 3 columns]
